<a href="https://colab.research.google.com/github/laresamdeola/Computer-Vision/blob/main/Parkinson's_Disease_Computer_Vision_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [74]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import keras

In [75]:
tf.__version__

'2.15.0'

In [76]:
import os
import shutil

In [77]:
os.getcwd()

'/content'

Folder Structure

In [ ]:
# --content
#   --Parkinson_Data
#     --test_data
#       --control
#       --pd
#     --train_data
#       --control
#       --pd

In [78]:
os.listdir('Parkinson_Data')

['test_data', 'train_data']

In [79]:
# Uncomment and Run this cell once

# shutil.rmtree('Parkinson_Data/.ipynb_checkpoints')
# shutil.rmtree('Parkinson_Data/train_data/.ipynb_checkpoints')
# shutil.rmtree('Parkinson_Data/test_data/.ipynb_checkpoints')

In [80]:
os.listdir('Parkinson_Data')

['test_data', 'train_data']

In [81]:
os.listdir('Parkinson_Data/train_data')
os.listdir('Parkinson_Data/test_data')

['control', 'pd']

In [82]:
os.listdir('Parkinson_Data/train_data')
os.listdir('Parkinson_Data/test_data')

['control', 'pd']

Image Augmentation - for preventing overfitting

In [83]:
# Image size increased to 124x124 because of the InceptionV3 Model (min. image size - 75x75) and Xception (min. image size 71x71)

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

train_set = train_datagen.flow_from_directory(
    'Parkinson_Data/train_data',
    target_size=(124,124),
    batch_size=32,
    class_mode='binary'
)

#

Found 34 images belonging to 2 classes.


In [84]:
print(train_set.labels)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [85]:
test_datagen = ImageDataGenerator(
    rescale=1./255
)

test_set = test_datagen.flow_from_directory(
    'Parkinson_Data/test_data',
    target_size=(124,124),
    batch_size=32,
    class_mode='binary'
)

Found 8 images belonging to 2 classes.


CNN Model - using the Keras Sequential Class

In [86]:
cnn = tf.keras.models.Sequential()

  Add the Convolutional Layer

In [87]:
cnn.add(tf.keras.layers.Conv2D(filters=32,
                               kernel_size=3,
                               activation='relu',
                               input_shape=[124,124,3]))

Add the Pooling Layer

In [88]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

Flatten the layer

In [89]:
cnn.add(tf.keras.layers.Flatten())

In [90]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

In [91]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

Compile the Neural Network

In [92]:
cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Train the Neural Network

In [93]:
# When you're training the whole dataset, change the epochs argument to 50 or 100 depending on the val_accuracy

cnn.fit(x=train_set, validation_data=test_set, epochs=10)

Epoch 1/10
2/2 [==============================] - 2s 387ms/step - loss: 1.0002 - accuracy: 0.5000 - val_loss: 0.5366 - val_accuracy: 0.7500
Epoch 2/10
2/2 [==============================] - 0s 198ms/step - loss: 0.5858 - accuracy: 0.7353 - val_loss: 2.3647 - val_accuracy: 0.6250
Epoch 3/10
2/2 [==============================] - 0s 55ms/step - loss: 0.7185 - accuracy: 0.7059 - val_loss: 2.5823 - val_accuracy: 0.6250
Epoch 4/10
2/2 [==============================] - 0s 193ms/step - loss: 0.7965 - accuracy: 0.6471 - val_loss: 2.3188 - val_accuracy: 0.6250
Epoch 5/10
2/2 [==============================] - 0s 106ms/step - loss: 0.6646 - accuracy: 0.6471 - val_loss: 1.8800 - val_accuracy: 0.6250
Epoch 6/10
2/2 [==============================] - 0s 195ms/step - loss: 0.3135 - accuracy: 0.8235 - val_loss: 1.6083 - val_accuracy: 0.5000
Epoch 7/10
2/2 [==============================] - 0s 51ms/step - loss: 0.3256 - accuracy: 0.8824 - val_loss: 1.4948 - val_accuracy: 0.6250
Epoch 8/10
2/2 [======

In [94]:
print(cnn.input_shape)

(None, 124, 124, 3)


In [95]:
cnn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 122, 122, 32)      896       
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 61, 61, 32)        0         
 g2D)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 119072)            0         
                                                                 
 dense_16 (Dense)            (None, 128)               15241344  
                                                                 
 dense_17 (Dense)            (None, 1)                 129       
                                                                 
Total params: 15242369 (58.15 MB)
Trainable params: 15242369 (58.15 MB)
Non-trainable params: 0 (0.00 Byte)
____________

In [96]:
print(train_set.num_classes)

2


In [97]:
import tensorflow

In [98]:
from keras.applications import ResNet50, VGG16, InceptionV3, Xception
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.optimizers import Adam

ResNet50

In [99]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(124, 124, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
resnet50_model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

resnet50_model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

In [100]:
# resnet50_model.summary()

In [101]:
print(resnet50_model.input)

KerasTensor(type_spec=TensorSpec(shape=(None, 124, 124, 3), dtype=tf.float32, name='input_8'), name='input_8', description="created by layer 'input_8'")


In [102]:
steps_per_epoch = max(1, train_set.samples // train_set.batch_size)
validation_steps = max(1, test_set.samples // test_set.batch_size)

In [103]:
history = resnet50_model.fit(
    train_set,
    steps_per_epoch=steps_per_epoch,
    validation_data=test_set,
    validation_steps=validation_steps,
    epochs=10
)

Epoch 1/10
1/1 [==============================] - 6s 6s/step - loss: 0.8953 - accuracy: 0.0000e+00 - val_loss: 0.8776 - val_accuracy: 0.5000
Epoch 2/10
1/1 [==============================] - 2s 2s/step - loss: 0.8649 - accuracy: 0.5312 - val_loss: 1.0298 - val_accuracy: 0.5000
Epoch 3/10
1/1 [==============================] - 0s 247ms/step - loss: 1.0664 - accuracy: 0.5000 - val_loss: 0.9534 - val_accuracy: 0.5000
Epoch 4/10
1/1 [==============================] - 0s 250ms/step - loss: 0.9334 - accuracy: 0.5312 - val_loss: 0.7919 - val_accuracy: 0.5000
Epoch 5/10
1/1 [==============================] - 0s 237ms/step - loss: 0.8257 - accuracy: 0.5000 - val_loss: 0.6550 - val_accuracy: 0.5000
Epoch 6/10
1/1 [==============================] - 0s 124ms/step - loss: 0.4486 - accuracy: 1.0000 - val_loss: 0.6317 - val_accuracy: 0.5000
Epoch 7/10
1/1 [==============================] - 0s 271ms/step - loss: 0.6601 - accuracy: 0.5312 - val_loss: 0.6170 - val_accuracy: 0.5000
Epoch 8/10
1/1 [======

VGG16

In [104]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(124, 124, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)  # Changed to 1 unit with sigmoid activation
vgg16_model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

vgg16_model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])  # Changed to 'binary_crossentropy'


In [105]:
history = vgg16_model.fit(
    train_set,
    steps_per_epoch=steps_per_epoch,
    validation_data=test_set,
    validation_steps=validation_steps,
    epochs=10
)

Epoch 1/10
1/1 [==============================] - 5s 5s/step - loss: 0.6950 - accuracy: 0.5938 - val_loss: 0.7054 - val_accuracy: 0.5000
Epoch 2/10
1/1 [==============================] - 1s 869ms/step - loss: 0.6520 - accuracy: 0.5000 - val_loss: 0.6060 - val_accuracy: 0.7500
Epoch 3/10
1/1 [==============================] - 0s 162ms/step - loss: 0.5655 - accuracy: 1.0000 - val_loss: 0.5981 - val_accuracy: 0.5000
Epoch 4/10
1/1 [==============================] - 0s 360ms/step - loss: 0.5370 - accuracy: 0.8125 - val_loss: 0.6026 - val_accuracy: 0.5000
Epoch 5/10
1/1 [==============================] - 0s 367ms/step - loss: 0.5118 - accuracy: 0.8438 - val_loss: 0.5999 - val_accuracy: 0.5000
Epoch 6/10
1/1 [==============================] - 0s 141ms/step - loss: 0.5906 - accuracy: 1.0000 - val_loss: 0.6484 - val_accuracy: 0.6250
Epoch 7/10
1/1 [==============================] - 0s 394ms/step - loss: 0.5080 - accuracy: 0.7188 - val_loss: 0.7247 - val_accuracy: 0.6250
Epoch 8/10
1/1 [=======

InceptionV3

In [106]:
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(124, 124, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)  # Changed to 1 unit with sigmoid activation
inceptionv3_model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

inceptionv3_model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])  # Changed to 'binary_crossentropy'

87910968/87910968 [==============================] - 1s 0us/step


In [107]:
history = inceptionv3_model.fit(
    train_set,
    steps_per_epoch=steps_per_epoch,
    validation_data=test_set,
    validation_steps=validation_steps,
    epochs=10
)

Epoch 1/10
1/1 [==============================] - 9s 9s/step - loss: 0.8715 - accuracy: 0.5312 - val_loss: 13.0303 - val_accuracy: 0.5000
Epoch 2/10
1/1 [==============================] - 1s 1s/step - loss: 1.5783e-15 - accuracy: 1.0000 - val_loss: 21.3552 - val_accuracy: 0.5000
Epoch 3/10
1/1 [==============================] - 0s 126ms/step - loss: 16.5164 - accuracy: 0.5000 - val_loss: 19.6622 - val_accuracy: 0.5000
Epoch 4/10
1/1 [==============================] - 0s 230ms/step - loss: 19.1913 - accuracy: 0.5000 - val_loss: 12.1236 - val_accuracy: 0.5000
Epoch 5/10
1/1 [==============================] - 0s 128ms/step - loss: 2.3699e-16 - accuracy: 1.0000 - val_loss: 6.7691 - val_accuracy: 0.6250
Epoch 6/10
1/1 [==============================] - 0s 225ms/step - loss: 3.8770 - accuracy: 0.6875 - val_loss: 2.3959 - val_accuracy: 0.7500
Epoch 7/10
1/1 [==============================] - 0s 141ms/step - loss: 1.1187 - accuracy: 0.5000 - val_loss: 1.0628 - val_accuracy: 0.7500
Epoch 8/10
1

Xception

In [108]:
base_model = Xception(weights='imagenet', include_top=False, input_shape=(124, 124, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)  # Changed to 1 unit with sigmoid activation
xception_model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

xception_model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])  # Changed to 'binary_crossentropy'


83683744/83683744 [==============================] - 1s 0us/step


In [109]:
history = xception_model.fit(
    train_set,
    steps_per_epoch=steps_per_epoch,
    validation_data=test_set,
    validation_steps=validation_steps,
    epochs=10
)

Epoch 1/10
1/1 [==============================] - 5s 5s/step - loss: 0.4942 - accuracy: 1.0000 - val_loss: 3.1413 - val_accuracy: 0.5000
Epoch 2/10
1/1 [==============================] - 0s 126ms/step - loss: 2.6010e-04 - accuracy: 1.0000 - val_loss: 5.1090 - val_accuracy: 0.5000
Epoch 3/10
1/1 [==============================] - 2s 2s/step - loss: 4.2884 - accuracy: 0.5312 - val_loss: 4.2125 - val_accuracy: 0.5000
Epoch 4/10
1/1 [==============================] - 0s 237ms/step - loss: 3.8673 - accuracy: 0.5312 - val_loss: 2.5494 - val_accuracy: 0.5000
Epoch 5/10
1/1 [==============================] - 0s 117ms/step - loss: 2.8065 - accuracy: 0.5000 - val_loss: 1.0404 - val_accuracy: 0.5000
Epoch 6/10
1/1 [==============================] - 0s 123ms/step - loss: 0.2621 - accuracy: 1.0000 - val_loss: 0.6983 - val_accuracy: 0.6250
Epoch 7/10
1/1 [==============================] - 0s 100ms/step - loss: 0.6238 - accuracy: 0.5000 - val_loss: 1.1485 - val_accuracy: 0.5000
Epoch 8/10
1/1 [======

Credits

In [110]:
# @misc{https://doi.org/10.5281/zenodo.13120260,
#     doi = {10.5281/ZENODO.13120260},
#     url = {https://zenodo.org/doi/10.5281/zenodo.13120260},
#     author = {Adeola, Lare},
#     keywords = {Computer Vision, Applied Machine Learning, Deep Learning, Parkinson's Disease, neurodegenerative disease},
#     title = {Using Computer Vision to detect Parkinson's Disease},
#     publisher = {Zenodo},
#     year = {2024},
#     copyright = {Creative Commons Attribution 4.0 International}
# }